In [1]:
#basic data science imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# py module
import prepare as p

#Stats imports
from scipy import stats
import math
from scipy.stats import pearsonr, spearmanr, ttest_1samp, ttest_ind, chi2_contingency


# Load CSVs

In [2]:
# Wiki CSV
wiki_df = pd.read_csv('battlebots_wiki.csv', index_col=0)
wiki_df.head(10)

,links,reboot_seasons,team,weapons,weight,other_events,power,names
1,https://battlebots.fandom.com/wiki/Aegis,World Championship V,Team Phalanx,Flipper,250lbs,BattleBots: Bounty Hunters,2x Castle 2028 brushless motors,Aegis
0,https://battlebots.fandom.com/wiki/Ambush,World Championship II,Blindside Robotics,Lifting forks,250lbs,NaN,NaN,Ambush
1,https://battlebots.fandom.com/wiki/Ambush,World Championship II,Blindside Robotics,Circular saws,250lbs,NaN,NaN,Ambush
2,https://battlebots.fandom.com/wiki/Ambush,World Championship II,Blindside Robotics,Flamethrower,250lbs,NaN,NaN,Ambush
5,https://battlebots.fandom.com/wiki/Annihilator,NaN,Team Brute Force,Overhead Spinning Blade,NaN,NaN,NaN,Annihilator
6,https://battlebots.fandom.com/wiki/Anubis,NaN,Team Hunter Robotics,Four Lifting Arms,NaN,NaN,NaN,Anubis
7,https://battlebots.fandom.com/wiki/ATOM94,World Championship V,AutoVoltz Robotics,Vertical spinner,NaN,BattleBots: Bounty Hunters,5450mAh 22.2V 120C LiPo batteries,ATOM94
0,https://battlebots.fandom.com/wiki/Axe_Backwards,World Championship III,Team Axe Backwards (Formerly KurTrox Robotix),Full body drum spinner,250lbs,BattleBots: Bounty Hunters,NaN,Axe_Backwards
1,https://battlebots.fandom.com/wiki/Axe_Backwards,World Championship IV,Team Axe Backwards (Formerly KurTrox Robotix),Full body drum spinner,250lbs,BattleBots: Bounty Hunters,NaN,Axe_Backwards
2,https://battlebots.fandom.com/wiki/Axe_Backwards,World Championship V,Team Axe Backwards (Formerly KurTrox Robotix),Full body drum spinner,250lbs,BattleBots: Bounty Hunters,NaN,Axe_Backwards


In [3]:
wiki_df.shape

(622, 8)

In [4]:
# Links CSV
links_csv = pd.read_csv('links.csv')
links_csv.head(3)

,https://battlebots.fandom.com/wiki/Template:2015_Competitors
0,https://battlebots.fandom.com/wiki/Template:20...
1,https://battlebots.fandom.com/wiki/Template:20...
2,https://battlebots.fandom.com/wiki/Template:20...


In [5]:
# Reorder Columns
wiki_df = wiki_df[['reboot_seasons','names','weapons','weight','power','team','other_events','links']]
wiki_df.head()

,reboot_seasons,names,weapons,weight,power,team,other_events,links
1,World Championship V,Aegis,Flipper,250lbs,2x Castle 2028 brushless motors,Team Phalanx,BattleBots: Bounty Hunters,https://battlebots.fandom.com/wiki/Aegis
0,World Championship II,Ambush,Lifting forks,250lbs,NaN,Blindside Robotics,NaN,https://battlebots.fandom.com/wiki/Ambush
1,World Championship II,Ambush,Circular saws,250lbs,NaN,Blindside Robotics,NaN,https://battlebots.fandom.com/wiki/Ambush
2,World Championship II,Ambush,Flamethrower,250lbs,NaN,Blindside Robotics,NaN,https://battlebots.fandom.com/wiki/Ambush
5,NaN,Annihilator,Overhead Spinning Blade,NaN,NaN,Team Brute Force,NaN,https://battlebots.fandom.com/wiki/Annihilator


In [6]:
# Load Stats History CSV
all_stats = pd.read_csv('all_stats_history.csv', index_col=0)
all_stats.head()

,robot_name,total_matches,win_percentage,total_wins,losses,ko,ko_percentage,avg_ko_time,ko_against,ko_against_percentage,decision_wins,season
0,axe_backwards,11.0,9%,1.0,10.0,1.0,9%,135 s,8.0,73%,0.0,2019
1,battlesaw,2.0,0%,0.0,2.0,0.0,0%,NaN,2.0,100%,0.0,2019
2,bite_force,24.0,96%,23.0,1.0,16.0,67%,100 s,0.0,0%,7.0,2019
3,black_dragon,24.0,67%,16.0,8.0,10.0,42%,90 s,0.0,0%,8.0,2019
4,blacksmith,20.0,40%,8.0,12.0,3.0,15%,166 s,2.0,10%,4.0,2019


In [7]:
# Copy of original
all_stats_copy = all_stats.copy()

In [8]:
all_stats.shape

(249, 12)

In [9]:
list(all_stats.columns)


['robot_name',
 'total_matches',
 'win_percentage',
 'total_wins',
 'losses',
 'ko',
 'ko_percentage',
 'avg_ko_time',
 'ko_against',
 'ko_against_percentage',
 'decision_wins',
 'season']

# Prep/Clean Data

In [10]:
# List to reorder columns
col_names = ['robot_name', 'season',
 'total_matches',
 'win_percentage',
 'total_wins',
 'losses',
 'ko',
 'ko_percentage',
 'avg_ko_time',
 'ko_against',
 'ko_against_percentage',
 'decision_wins'
 ]

In [11]:
# Reorder columns
all_stats = all_stats[['robot_name', 'season',
 'total_matches',
 'win_percentage',
 'total_wins',
 'losses',
 'ko',
 'ko_percentage',
 'avg_ko_time',
 'ko_against',
 'ko_against_percentage',
 'decision_wins']]
all_stats.head()

,robot_name,season,total_matches,win_percentage,total_wins,losses,ko,ko_percentage,avg_ko_time,ko_against,ko_against_percentage,decision_wins
0,axe_backwards,2019,11.0,9%,1.0,10.0,1.0,9%,135 s,8.0,73%,0.0
1,battlesaw,2019,2.0,0%,0.0,2.0,0.0,0%,NaN,2.0,100%,0.0
2,bite_force,2019,24.0,96%,23.0,1.0,16.0,67%,100 s,0.0,0%,7.0
3,black_dragon,2019,24.0,67%,16.0,8.0,10.0,42%,90 s,0.0,0%,8.0
4,blacksmith,2019,20.0,40%,8.0,12.0,3.0,15%,166 s,2.0,10%,4.0


In [12]:
# Check for same bot for multiple seasons
all_stats.robot_name.value_counts()

ghost_rapto       2
rotator           2
hijin             2
gruf              2
valkyri           2
                 ..
tombstone         1
texas_twister     1
subzero           1
son_of_whyachi    1
witch_doctor      1
Name: robot_name, Length: 178, dtype: int64

In [13]:
# Check for Nulls
all_stats.isna().sum()

robot_name                1
season                    0
total_matches             0
win_percentage            2
total_wins                0
losses                    0
ko                        0
ko_percentage             2
avg_ko_time              39
ko_against                0
ko_against_percentage     2
decision_wins             0
dtype: int64

In [14]:
# Normalize percentage columns and change dtype 
all_stats.win_percentage = all_stats.win_percentage.str.replace('%','').astype(float)
all_stats.ko_percentage = all_stats.ko_percentage.str.replace('%','').astype(float)
all_stats.ko_against_percentage = all_stats.ko_against_percentage.str.replace('%','').astype(float)
all_stats.head()

,robot_name,season,total_matches,win_percentage,total_wins,losses,ko,ko_percentage,avg_ko_time,ko_against,ko_against_percentage,decision_wins
0,axe_backwards,2019,11.0,9.0,1.0,10.0,1.0,9.0,135 s,8.0,73.0,0.0
1,battlesaw,2019,2.0,0.0,0.0,2.0,0.0,0.0,NaN,2.0,100.0,0.0
2,bite_force,2019,24.0,96.0,23.0,1.0,16.0,67.0,100 s,0.0,0.0,7.0
3,black_dragon,2019,24.0,67.0,16.0,8.0,10.0,42.0,90 s,0.0,0.0,8.0
4,blacksmith,2019,20.0,40.0,8.0,12.0,3.0,15.0,166 s,2.0,10.0,4.0


In [15]:
# Function to turn percentage columns into decimals

def convert_percent_cols(df):
    
    # Remove '%' and convert to Float
    df.win_percentage = df.win_percentage.str.replace('%','').astype(float)
    df.ko_percentage = df.ko_percentage.str.replace('%','').astype(float)
    df.ko_against_percentage = df.ko_against_percentage.str.replace('%','').astype(float)
    
    # Lists to hold numbers
    win_percentage_list = []
    ko_percentage_list = []
    ko_against_percentage_list = []
    
    # Loop for win percentage
    for number in df.win_percentage:
        
        # Multiply to turn into decimal
        new = number * 0.01
        
        # Append lis
        win_percentage_list.append(new)
    
    # Loop for KO percentage
    for number in df.ko_percentage:
        
        # Multiply to turn into decimal
        new = number * 0.01
        
        # Append lis
        ko_percentage_list.append(new)
    
    # Loop for KO against percentage
    for number in df.ko_against_percentage:
        
        # Multiply to turn into decimal
        new = number * 0.01
        
        # Append lis
        ko_against_percentage_list.append(new)
        
    df['win_percentage'] = win_percentage_list
    df['ko_percentage'] = ko_percentage_list
    df['ko_against_percentage'] = ko_against_percentage_list
    
    return df


In [16]:
# Test function on copy df
test = convert_percent_cols(all_stats_copy)
test.head()

,robot_name,total_matches,win_percentage,total_wins,losses,ko,ko_percentage,avg_ko_time,ko_against,ko_against_percentage,decision_wins,season
0,axe_backwards,11.0,0.09,1.0,10.0,1.0,0.09,135 s,8.0,0.73,0.0,2019
1,battlesaw,2.0,0.00,0.0,2.0,0.0,0.00,NaN,2.0,1.00,0.0,2019
2,bite_force,24.0,0.96,23.0,1.0,16.0,0.67,100 s,0.0,0.00,7.0,2019
3,black_dragon,24.0,0.67,16.0,8.0,10.0,0.42,90 s,0.0,0.00,8.0,2019
4,blacksmith,20.0,0.40,8.0,12.0,3.0,0.15,166 s,2.0,0.10,4.0,2019


In [17]:
# Function to remove 's' and covert to INT

def convert_sec_col(df):
    

    # Remove 's' 
    df.avg_ko_time = df.avg_ko_time.str.replace('s','')
    
    # Replace NaN with '0'
    df.avg_ko_time = df.avg_ko_time.fillna(0)
    
    df.avg_ko_time = df.avg_ko_time.astype(float)
    
    return df

In [18]:
test = convert_sec_col(test)
test.head()

,robot_name,total_matches,win_percentage,total_wins,losses,ko,ko_percentage,avg_ko_time,ko_against,ko_against_percentage,decision_wins,season
0,axe_backwards,11.0,0.09,1.0,10.0,1.0,0.09,135.0,8.0,0.73,0.0,2019
1,battlesaw,2.0,0.00,0.0,2.0,0.0,0.00,0.0,2.0,1.00,0.0,2019
2,bite_force,24.0,0.96,23.0,1.0,16.0,0.67,100.0,0.0,0.00,7.0,2019
3,black_dragon,24.0,0.67,16.0,8.0,10.0,0.42,90.0,0.0,0.00,8.0,2019
4,blacksmith,20.0,0.40,8.0,12.0,3.0,0.15,166.0,2.0,0.10,4.0,2019


In [19]:
# Use cleaning function on working df
all_stats = convert_sec_col(all_stats)
all_stats.head()

,robot_name,season,total_matches,win_percentage,total_wins,losses,ko,ko_percentage,avg_ko_time,ko_against,ko_against_percentage,decision_wins
0,axe_backwards,2019,11.0,9.0,1.0,10.0,1.0,9.0,135.0,8.0,73.0,0.0
1,battlesaw,2019,2.0,0.0,0.0,2.0,0.0,0.0,0.0,2.0,100.0,0.0
2,bite_force,2019,24.0,96.0,23.0,1.0,16.0,67.0,100.0,0.0,0.0,7.0
3,black_dragon,2019,24.0,67.0,16.0,8.0,10.0,42.0,90.0,0.0,0.0,8.0
4,blacksmith,2019,20.0,40.0,8.0,12.0,3.0,15.0,166.0,2.0,10.0,4.0


# Split into Train, Validate, Test sets

In [ ]:
# Split Data
t